In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.signal import savgol_filter

# Main directory containing the folders
main_dir = r"C:\Data\RawData"

# List of subfolders to process
subfolders = [
    "Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall",
    "Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall",
    "Base_Isolated_Moment_Frame_without_Impact",
    "Fixed_Base_Moment_Frame"
]

# Smoothing parameters for each displacement 
smoothing_params = {
    "s2": (201, 3),
    "s3": (201, 3),
    "s4": (201, 3),
    "s5": (201, 3),
    "dWestWall": (51, 2),
    "dEastWall": (51, 2)
}

# Specified parameters to keep
parameters_to_keep = ["Time", "aextx", "s2", "s3", "s4", "s5", "s16", "s17", "s18", "s19", "lc1", "lc2", "lc3", "lc4"]

# Function to process a single ASC file
def process_asc_file(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Skipping the first four rows 
    column_names = lines[4].strip().split()
    units = lines[5].strip().split()

    # Loading the data starting from the 7th row
    data = pd.read_csv(input_file, sep=r'\s+', skiprows=6, names=column_names)

    # Filtering existing columns
    existing_columns = [col for col in parameters_to_keep if col in data.columns]

    # Handling FCEast (between lc1 and lc2)
    if "lc1" in data.columns and "lc2" in data.columns:
        peak_ic1 = data["lc1"].abs().max()
        peak_ic2 = data["lc2"].abs().max()
        selected = data["lc1"] if peak_ic1 >= peak_ic2 else data["lc2"]
        if selected.max() < abs(selected.min()):  # Negative peak dominates
            selected = -selected
        data["FCEast"] = selected

    # Handling FCWest (between lc3 and lc4)
    if "lc3" in data.columns and "lc4" in data.columns:
        peak_ic3 = data["lc3"].abs().max()
        peak_ic4 = data["lc4"].abs().max()
        selected = data["lc3"] if peak_ic3 >= peak_ic4 else data["lc4"]
        if selected.max() < abs(selected.min()):  # Negative peak dominates
            selected = -selected
        data["FCWest"] = selected
        
    # Adjusting FC by adding the absolute of the average value (only if average is negative)
    if "FCEast" in data.columns:
        fc_average = data["FCEast"].mean()
        if fc_average < 0:
            data["FCEast"] = data["FCEast"] + abs(fc_average)
        elif fc_average > 0:
            data["FCEast"] = data["FCEast"] - abs(fc_average)
            
    if "FCWest" in data.columns:
        fc_average = data["FCWest"].mean()
        if fc_average < 0:
            data["FCWest"] = data["FCWest"] + abs(fc_average)    
        elif fc_average > 0:
            data["FCWest"] = data["FCWest"] - abs(fc_average)   

    
    # Handling s16-s17 (West Wall)
    if "s16" in data.columns and "s17" in data.columns:
        data["dWestWall"] = data[["s16", "s17"]].max(axis=1)
    elif "s16" in data.columns:
        data["dWestWall"] = data["s16"]
    elif "s17" in data.columns:
        data["dWestWall"] = data["s17"]
    
    # Handling s18-s19 (East Wall)
    if "s18" in data.columns and "s19" in data.columns:
        data["dEastWall"] = -data[["s18", "s19"]].min(axis=1)
    elif "s18" in data.columns:
        data["dEastWall"] = -data["s18"]
    elif "s19" in data.columns:
        data["dEastWall"] = -data["s19"]


    # Custom smoothing for each displacement
    def calculate_smoothed_velocity_acceleration(displacement, time, window_length, poly_order):
        smoothed_disp = savgol_filter(displacement, window_length=window_length, polyorder=poly_order)
        velocity = np.gradient(smoothed_disp, time)
        smoothed_velocity = savgol_filter(velocity, window_length=window_length, polyorder=poly_order)
        acceleration = np.gradient(smoothed_velocity, time) / 386.09  # in g
        return smoothed_velocity, acceleration

    # Calculating velocities and accelerations
    for col, (window_length, poly_order) in smoothing_params.items():
        if col in data.columns:
            vel, acc = calculate_smoothed_velocity_acceleration(data[col], data["Time"], window_length, poly_order)
            data[f"V{col}"] = vel
            data[f"ACC{col}"] = acc

    # Organizing final columns
    final_columns = ["Time", "aextx", "s2", "s3", "s4", "s5", "dWestWall", "dEastWall"]
    final_columns += [f"V{col}" for col in smoothing_params.keys()]
    final_columns += [f"ACC{col}" for col in smoothing_params.keys()]
    if "FCEast" in data.columns:
        final_columns.append("FCEast")
    if "FCWest" in data.columns:
        final_columns.append("FCWest")

    existing_columns = [col for col in final_columns if col in data.columns]

    # Creating the filtered dataframe
    filtered_data = data[existing_columns].round(6)

    # Adjusting units
    filtered_units = []
    for col in existing_columns:
        if col == "Time":
            filtered_units.append("s")
        elif col == "aextx":
            filtered_units.append(units[column_names.index(col)])
        elif "V" in col:
            filtered_units.append("in/s")
        elif "ACC" in col:
            filtered_units.append("g")
        elif col in ["FCEast", "FCWest"]:
            filtered_units.append("Kip")
        else:
            filtered_units.append("in")

    # Writing the output file
    with open(output_file, 'w') as out_file:
        out_file.write('\t'.join(existing_columns) + '\n')
        out_file.write('\t'.join(filtered_units) + '\n')
        filtered_data.to_csv(out_file, sep='\t', index=False, header=False, float_format="%.6f")

    print("Processed:", input_file)

# Looping through each subfolder and processing all ASC files
for folder in subfolders:
    folder_path = os.path.join(main_dir, folder)
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".asc"):
                input_file = os.path.join(root, file)
                output_file = os.path.join(root, f"filtered_{file.replace('.asc', '.txt')}")
                process_asc_file(input_file, output_file)

print("Processing completed for all ASC files.")
